In [1]:
import sys, pickle, json, os, re
import numpy as np
from collections import defaultdict
from pprint import pprint
from typing import List, Dict, Any
sys.path.append('../src/')
sys.path.append("../../scripts/")
from utils import find_repo_root

import matplotlib.pyplot as plt
from plt_config import use_latex, TU_GREY, TU_RED
use_latex()
project_root_dir = find_repo_root()
print("Project root directory:", project_root_dir)

Project root directory: /home/etschgi1/REPOS/Masterarbeit


In [2]:
ray_folder = "/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_full_testing/tune_logs/MGNN_hyp_small_full_full_mat_loss.py"
trials = []
for folder in os.listdir(ray_folder):
    sub_folder_path = os.path.join(ray_folder, folder)
    if os.path.exists(os.path.join(sub_folder_path, "eval_res.json")):
        with open(os.path.join(sub_folder_path, "eval_res.json")) as f:
            eval_res = json.load(f)
        with open(os.path.join(sub_folder_path, "params.json")) as f:
            params = json.load(f)
        with open(os.path.join(sub_folder_path, "result.json")) as f:
            last_text = f.readlines()[-1]
            epochs_trained = re.search('epoch": [0-9]*', last_text).group(0).split(": ")[-1]
        params["folder"] = folder
        params["epochs_trained"] = int(epochs_trained)
        trials.append((params, eval_res))


In [3]:
trials.sort(key=lambda x:x[1]["cummulative_stats"]["iterations"]["mean"])
print(f"Found {len(trials)} trials in {ray_folder}")
for trial in trials: 
    print(trial[0]["folder"])
    print(trial[1]["cummulative_stats"])
    print(trial[0]["epochs_trained"])
    print("---")


Found 4 trials in /home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_full_testing/tune_logs/MGNN_hyp_small_full_full_mat_loss.py
hyperopt_train_036e6_00001_1_batch_size=16,data_aug_factor=2,edge_threshold_val=3.5698,hidden_dim=128,lr=0.0009,message_net_dropou_2025-07-18_09-31-42
{'energy_abs': {'mean': -1086.2075915618307, 'std': 271.67350948402606}, 'iterations': {'mean': 12.22, 'std': 1.6886681142249353}, 'energy_rel': {'mean': -0.6920868749607015, 'std': 0.03634697261424198}, 'diis': {'mean': 0.20714547915559142, 'std': 0.02992177985057091}, 'rmse': {'mean': 0.0094223677499698, 'std': 0.0028106254813412036}}
5
---
hyperopt_train_036e6_00096_96_batch_size=8,data_aug_factor=1,edge_threshold_val=2.9918,hidden_dim=256,lr=0.0001,message_net_dropou_2025-07-18_16-58-59
{'energy_abs': {'mean': -1041.408998322332, 'std': 258.55326992073867}, 'iterations': {'mean': 12.36, 'std': 2.02741214359587}, 'energy_rel': {'mean': -0.6641685891496949, 'std': 0.03239900244105729}, 'diis': {

In [4]:
top = 10
table_data = []
for i, (params, metrics) in enumerate(trials[:top]):
    metrics = metrics["cummulative_stats"]
    row = {
        'ID': i,
        'Iterations': f"{float(metrics['iterations']['mean']):.1f}({float(metrics['iterations']['std']):.1f})",
        r"$\Delta E_\text{HF}$": f"{float(metrics['energy_abs']['mean']):.1f}({float(metrics['energy_abs']['std']):.1f})",
        r"$\delta E_\text{HF}$": f"{float(metrics['energy_rel']['mean']):.3f}({float(metrics['energy_rel']['std']):.3f})",
        'DIIS error': f"{float(metrics['diis']['mean']):.3f}({float(metrics['diis']['std']):.3f})",
        r"$\overline{\text{RMSE}}$": f"{float(metrics['rmse']['mean']):.4f}({float(metrics['rmse']['std']):.4f})"
    }
    table_data.append(row)

import pandas as pd
df = pd.DataFrame(table_data)

latex = df.to_latex(index=False)
print(latex)

\begin{tabular}{rlllll}
\toprule
ID & Iterations & $\Delta E_\text{HF}$ & $\delta E_\text{HF}$ & DIIS error & $\overline{\text{RMSE}}$ \\
\midrule
0 & 12.2(1.7) & -1086.2(271.7) & -0.692(0.036) & 0.207(0.030) & 0.0094(0.0028) \\
1 & 12.4(2.0) & -1041.4(258.6) & -0.664(0.032) & 0.190(0.029) & 0.0092(0.0026) \\
2 & 12.7(3.1) & -1082.0(274.2) & -0.689(0.039) & 0.209(0.035) & 0.0099(0.0034) \\
3 & 13.0(2.7) & -1071.9(281.1) & -0.681(0.050) & 0.312(0.772) & 0.0102(0.0036) \\
\bottomrule
\end{tabular}



In [5]:
table_data = []
for i, (params, metrics) in enumerate(trials[:top]):
    metrics = metrics["cummulative_stats"]
    row = {
        'Parameter': int(i),
        'Hidden Dimension': int(params["hidden_dim"]),
        'Batch Size': int(params["batch_size"]),
        "Data aug.": f'{float(params["data_aug_factor"]):.2f}',
        "Edge thres.": f'{float(params["edge_threshold_val"]):.2f}',
        "Message p. steps": int(params["message_passing_steps"]),
        "Message Net Dropout": f'{float(params["message_net_dropout"]):.2f}',
        "Message Net Layers": int(params["message_net_layers"]),
        "Learn Rate": f'{float(params["lr"]):.2e}',
        "Weight Decay": f'{float(params["weight_decay"]):.2e}',
    }
    table_data.append(row)

import pandas as pd
df = pd.DataFrame(table_data).T


latex = df.to_latex(index=True)
print(latex)

\begin{tabular}{lllll}
\toprule
 & 0 & 1 & 2 & 3 \\
\midrule
Parameter & 0 & 1 & 2 & 3 \\
Hidden Dimension & 128 & 256 & 512 & 512 \\
Batch Size & 16 & 8 & 32 & 32 \\
Data aug. & 2.00 & 1.00 & 2.00 & 4.00 \\
Edge thres. & 3.57 & 2.99 & 2.46 & 3.96 \\
Message p. steps & 3 & 4 & 5 & 3 \\
Message Net Dropout & 0.27 & 0.02 & 0.07 & 0.01 \\
Message Net Layers & 3 & 4 & 2 & 3 \\
Learn Rate & 9.04e-04 & 1.27e-04 & 1.32e-04 & 1.22e-04 \\
Weight Decay & 2.13e-05 & 1.45e-05 & 1.18e-04 & 7.85e-05 \\
\bottomrule
\end{tabular}

